# Alignment with bigstream
---

BigStream is a library of tools for image registration of huge images. It uses big data tools like Zarr and DASK to enable working with too-large-for-memory datasets and to make costly alignments finish in practical amounts of time by distributing the work on your workstation or cluster.

This tutorial will walk you through the following steps:

    1. Reading image data and metadata using Zarr
    2. Global affine alignment
    3. Piecewise affine alignments
    4. Piecewise deformable alignments

## Preliminary
---

Make sure BigStream is installed: `pip install bigstream`

You should also get the source code, which is located here: https://github.com/GFleishman/bigstream. \
Follow the instrucions on github to clone the repository, which contains the example data used for this tutorial.

## Tutorial data
---

In the BigStream repository the `resources` folder contains two images in N5 format.\
We will first access the data and metadata in these files using Zarr, which was installed with BigStream.

In [ ]:
# import library for reading
import zarr

# file paths to tutorial data N5 files
fix_path = '/groups/scicompsoft/home/fleishmang/source/bigstream/resources/fix.n5'
mov_path = '/groups/scicompsoft/home/fleishmang/source/bigstream/resources/mov.n5'

# create Zarr file object using N5Stores
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')

`fix_zarr` and `mov_zarr` are just lazy pointers to the N5 files; no image data has been loaded into memory yet.\
The first alignment step, global affine, only needs low resolution data; which we assume fits into available memory. \
You'll see later how to deal with images that do not fit into memory.

In [ ]:
# we'll need numpy now
import numpy as np

# get pointers to the low res scale level
# still just pointers, no data loaded into memory yet
fix_lowres = fix_zarr['/lowres']
mov_lowres = mov_zarr['/lowres']

# we need the voxel spacings for the low res data sets
# we can compute them from the low res data set metadata
fix_meta = fix_lowres.attrs.asdict()
mov_meta = mov_lowres.attrs.asdict()
fix_lowres_spacing = np.array(fix_meta['pixelResolution']) * fix_meta['downsamplingFactors']
mov_lowres_spacing = np.array(mov_meta['pixelResolution']) * mov_meta['downsamplingFactors']

# read data into memory as numpy arrays
# Why transpose? Images were stored as zyx, but we prefer xyz (metadata is already xyz)
fix_lowres_data = fix_lowres[...].transpose(2, 1, 0)
mov_lowres_data = mov_lowres[...].transpose(2, 1, 0)

# sanity check: print the voxel spacings and lowres dataset shapes
print(fix_lowres_data.shape, fix_lowres_spacing)
print(mov_lowres_data.shape, mov_lowres_spacing)

## Global affine alignment
---

The affine alignment algorithm is composed of three steps:

    1. Key point extraction from fixed and moving images
    2. Correspondence matching between fixed and moving key points using neighborhood correlation
    3. Affine alignment of corresponding key points using RANSAC

But this is all accomplished with one function call. The following cell will take some time to run:

In [ ]:
# affine alignment functions are in bigstream.affine
from bigstream import affine

# see below for explanation of parameters
global_affine = affine.ransac_affine(
    fix_lowres_data, mov_lowres_data,
    fix_lowres_spacing, mov_lowres_spacing,
    min_radius=6, max_radius=20, match_threshold=0.75,
)

# sanity check: print the result
print(global_affine)

# For tutorial data, should be approximately:
# [[ 9.91070543e-01  3.40205967e-02 -5.63125159e-03 -8.12590407e+01]
#  [-3.86715664e-02  1.02023436e+00 -6.30453307e-03 -1.33468687e+01]
#  [ 1.19321249e-02 -2.21371673e-02  9.67441910e-01 -2.42373194e-01]
#  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

### Parameters and output

    min_radius: radius in voxels of smallest expected blob/cell size
    
    max_radius: radius in voxels of largest expected blob/cell size
    
    match_threshold: neighborhood correlation between two key points must exceed this value for it to be a valid match
    
Other optional parameters are:
    
    cc_radius: key points are matched using correlation of the data in their neighborhoods, this is the neighborhood radius in voxels (default: 12)
    
    nspots: the maximum number of key point pairs to use to compute the affine alignment (default: 5000)
    
    align_threshold: points are considered aligned by the affine if they are less than this value apart, in micrometers (default: 2.0)
    
    num_sigma_max: the maximum number of filters to run in the blob detection (default: 10)
    
The return value is:
    
    global_affine: the return value is a 4x4 numpy array containing an affine transform matrix; this describes correspondence between points in the fixed image and moving image.

### Applying global affine

The alignment only gave us the affine transform matrix. Here we apply it to the moving image:

In [ ]:
# functions for applying transforms are in bigstream.transform
from bigstream import transform

# apply the global affine to the moving image
mov_lowres_aligned = transform.apply_global_affine(
    fix_lowres_data, mov_lowres_data,
    fix_lowres_spacing, mov_lowres_spacing,
    global_affine,
)

Now we can compare the fixed, moving, and aligned data. Try running this cell a few times with different values for `slc`.

In [ ]:
# we'll visualize the results with some image plots
from matplotlib import pyplot as plt

# plot some image slices to check on things
slc = 70
f_slc = fix_lowres_data[..., slc]
a_slc = mov_lowres_aligned[..., slc]
m_slc = mov_lowres_data[..., slc]

# normalize for display
f_slc = f_slc.astype(np.float32) / f_slc.max()
a_slc = a_slc.astype(np.float32) / a_slc.max()
m_slc = m_slc.astype(np.float32) / m_slc.max()

# make RGB versions
f_rgb = np.zeros(f_slc.shape + (3,))
f_rgb[..., 0] = f_slc * 2
a_rgb = np.zeros(a_slc.shape + (3,))
a_rgb[..., 0] = f_slc * 2
a_rgb[..., 1] = a_slc * 2
m_rgb = np.zeros(m_slc.shape + (3,))
m_rgb[..., 1] = m_slc * 2

# create figure and subplots
fig = plt.figure(figsize=(12,24))
fig.add_subplot(1, 3, 1)
plt.imshow(f_rgb)
fig.add_subplot(1, 3, 2)
plt.imshow(f_rgb)
plt.imshow(a_rgb)
fig.add_subplot(1, 3, 3)
plt.imshow(m_rgb)
plt.show()

## Local affine alignments
---

The tutorial dataset is small, so the global affine lines up almost all of the data well. But with a real dataset we would likely need to refine the global alignment with local alignments. In local affine alignment, the images are carved into overlapping tiles and a separate affine is computed for each tile. For large data sets there can be many tiles. To make this process tractable a cluster is constructed using [ClusterWrap](https://github.com/GFleishman/ClusterWrap) and DASK and the local affines are all computed in parallel.


The `prepare_piecewise_ransac_affine` function API is the same as `ransac_affine`, but with one new required argument:

    blocksize: iterable, length equal to the image dimension. The size of tiles in voxels.

In [ ]:
# Note use of mov_lowres_aligned as moving image rather than mov_lowres_data
# Note also that fix_lowres_spacing is used as the "moving" voxel spacing here
local_affines = affine.prepare_piecewise_ransac_affine(
    fix_lowres_data, mov_lowres_aligned,
    fix_lowres_spacing, fix_lowres_spacing,
    min_radius=6, max_radius=20, match_threshold=0.75,
    blocksize=[128,]*3,
)


# not a numpy array
print(type(local_affines))

# the first three dimensions index over the tiles
# the last two dimensions are the 4x4 affine matrices for each tile
print(local_affines.shape)

Notice the function returned immediately and did not return a numpy array. Any function in `bigstream` that begins with `prepare_` does not actually compute anything and does not return a concrete result. Instead, it sets up all the necessary relationships between functions and data to compute the result _some time later_. You can think of `prepare_` functions as returning a plan with a promise to compute later.

## Executing a prepared computation
---
Prepared computations can be executed on your local workstation or on a cluster with the `execute` function. The two cases are nearly identical, but we'll go over them separately.

### Workstation
(If you're working on a cluster you can skip this section)

BigStream is designed to work with very large datasets and every workstation has different (limited) resources and usage burden. When executing on a workstation it's important to think about:

    n_workers: the number of python processes to run in parallel
    
    threads_per_worker: the number of threads each process will run in parallel
    
    memory_limit: the maximum amount of RAM each worker can use
    
    config: changes to the dask configuration

The default value of `None` will cause BigStream to determine its resource limits automatically and if necessary it will max out your machine. To limit BigStream's access `n_workers` and `threads_per_worker` should be set based on how many cpu cores your workstation has and what other work the computer will be doing alongside bigstream. `memory_limit` is a threshold that if any individual worker crosses, the worker gets shutdown. `config` changes should really only be necessary if you run into problems, but here is a [list of dask configuration settings](https://docs.dask.org/en/latest/configuration-reference.html).

Basically, when doing big data computations, you need to be knowledgeable about your workstation's resources, the size of your data, and how to monitor resource consumption of an execution using `task manager`, `activity monitor`, `top` or similar.

In [ ]:
# function for executing prepared computations
from bigstream.cluster import execute

# execute the prepared computation, specifying resource limitations
local_affines = execute(
    local_affines,
    n_workers=None,
    threads_per_worker=None,
    memory_limit=None,
    config={},
)

# now we have a numpy array
print(type(local_affines))
print(local_affines.shape)

### Cluster
(If you're working on a workstation you can skip this section)

If your cluster manager is LSF then you're in luck, `bigstream` should work for you. If your cluster manager is not LSF, then don't worry. Adding support for additional cluster managers is a high priority and bigstream has been designed to make this very easy. Please [post an issue on the repository](https://github.com/GFleishman/bigstream/issues) and indicate which cluster manager you'd like support for.

On a cluster you have practically unlimited resources. BigStream will dynamically allocate workers between specified limits based on the size of the computation you give it. The resource limits you specify impact your time-to-result and cost. 

    cores: the number of cpu cores each worker gets
    
    processes: the number of python processes to run on each worker
    
    min_workers: minimum number of workers to maintain
    
    max_workers: the maximum number of workers to maintain
    
    config: changes to the dask configuration

BigStream will parallelize at the level of workers, processes, _and_ threads - so you have a lot of flexibility for setting up your computation. The default settings below are a good place to start. As your jobs get bigger you can increase things like `min_workers` and `max_workers`.

In [ ]:
# function for executing prepared computations
from bigstream.cluster import execute

# execute the prepared computation, specifying resource limitations
local_affines = execute(
    local_affines,
    cores=4,
    processes=1,
    min_workers=1,
    max_workers=4,
    config={},
)

# now we have a numpy array
print(type(local_affines))
print(local_affines.shape)

## Applying local affines
---

The `prepare_apply_local_affines` function API is the same as `apply_global_affine`, but with one new required argument and one new optional argument:

    blocksize: iterable, length equal to the image dimension. The size of tiles in voxels. Must be the same as the blocksize used to learn the local affines.
    
    global_affine (optional): a global affine matrix to apply before the local affines

In [ ]:
# apply the local affines to the moving image
#   Note we're using mov_lowres_data again - it's better
#   to provide the global and local affines together. They
#   are composed into a single transform - that way the moving
#   image is only resampled one time.
mov_lowres_aligned = transform.prepare_apply_local_affines(
    fix_lowres_data, mov_lowres_data,
    fix_lowres_spacing, mov_lowres_spacing,
    local_affines,
    blocksize=[128,]*3,
    global_affine=global_affine,
)

# prepared computation, so not a numpy array yet
print(type(mov_lowres_aligned))
print(mov_lowres_aligned.shape)

In [ ]:
# execute using defaults
# CONSIDER if you need to change resource parameters as discussed in "Executing a prepared compuation" section above
mov_lowres_aligned = execute(mov_lowres_aligned)

Similar to before, we can inspect the alignment:

In [ ]:
# plot some image slices to check on things
slc = 70
f_slc = fix_lowres_data[..., slc]
a_slc = mov_lowres_aligned[..., slc]
m_slc = mov_lowres_data[..., slc]

# normalize for display
f_slc = f_slc.astype(np.float32) / f_slc.max()
a_slc = a_slc.astype(np.float32) / a_slc.max()
m_slc = m_slc.astype(np.float32) / m_slc.max()

# make RGB versions
f_rgb = np.zeros(f_slc.shape + (3,))
f_rgb[..., 0] = f_slc * 2
a_rgb = np.zeros(a_slc.shape + (3,))
a_rgb[..., 0] = f_slc * 2
a_rgb[..., 1] = a_slc * 2
m_rgb = np.zeros(m_slc.shape + (3,))
m_rgb[..., 1] = m_slc * 2

# create figure and subplots
fig = plt.figure(figsize=(12,24))
fig.add_subplot(1, 3, 1)
plt.imshow(f_rgb)
fig.add_subplot(1, 3, 2)
plt.imshow(f_rgb)
plt.imshow(a_rgb)
fig.add_subplot(1, 3, 3)
plt.imshow(m_rgb)
plt.show()

## Working with higher resolution data
---

The final step is deformable alignment which can take advantage of high resolution features. Here we assume the higher resolution data is too large for memory and/or would be intractably slow to work on without parallelization. This section shows how BigStream's use of distributed computing enables you to align your data despite of these difficulties.

Compare the cell below to the second cell in the _Tutorial Data_ section. Note that here we do not read the data into memory but proceed with the lazy `zarr.Array` objects.

In [ ]:
# get pointers to the high res scale level
# still just pointers, no data loaded into memory
fix_highres = fix_zarr['/highres']
mov_highres = mov_zarr['/highres']

# we need the voxel spacings for the high res data sets
# we can compute them from the high res data set metadata
fix_meta = fix_highres.attrs.asdict()
mov_meta = mov_highres.attrs.asdict()
fix_highres_spacing = np.array(fix_meta['pixelResolution']) * fix_meta['downsamplingFactors']
mov_highres_spacing = np.array(mov_meta['pixelResolution']) * mov_meta['downsamplingFactors']

# sanity check: print the voxel spacings and lowres dataset shapes
print(fix_highres_spacing, mov_highres_spacing)
print(fix_highres.shape, mov_highres.shape)

Note the voxel spacings are in xyz order but the lazy array shapes are still zyx.


Before we can deform, we to apply the global and local affines to this high res version of the data. Compare the cell below to the first cell in the _Apply local affines_ subsection. First note we provide the highres lazy zarr.Arrays as inputs. We happen to know they are 2x larger along each axis - so importantly the `blocksize` parameter has been doubled. Also note we're using an additional optional parameter here:

    transpose: List of 3 boolean values. Should we transpose the axis order of the fixed image, moving image, or transform data.

In this case, since the fixed and moving image data are being read from zarr files, they must be transposed - but because the transform is being constructed from the global and local affine matrices, it does not need to be transposed.

In [ ]:
# apply the affines to the highres moving image
mov_highres_aligned = transform.prepare_apply_local_affines(
    fix_highres, mov_highres,
    fix_highres_spacing, mov_highres_spacing,
    local_affines,
    blocksize=[256,]*3,
    global_affine=global_affine,
    transpose=[True, True, False],
)

# sanity check
print(type(mov_highres_aligned))
print(mov_highres_aligned.shape)

In [ ]:
# this time, we'll write the aligned data to disk
write_path = './mov_highres_affine_aligned.zarr'

# execute with defaults
mov_highres_aligned = execute(mov_highres_aligned, write_path=write_path)

Note the use of `write_path` in `execute`. We've assumed the high resolution data is too large to fit into memory, so instead of computing the result and returning it to the memory of the local process, we're elected to write the result to disk as a zarr file and return a reference to the `zarr.Array` object.

Ensure the application was successful:

In [ ]:
# plot some image slices to check on things
#   Note for highres data we doubled the slice number
#   The fix/mov images still need to be transposed,
#   But mov_highres_aligned, which we made in the previous step
#   was written out in xyz order
slc = 140
f_slc = fix_highres[slc, ...].transpose(1,0)
a_slc = mov_highres_aligned[..., slc]
m_slc = mov_highres[slc, ...].transpose(1,0)

# normalize for display
f_slc = f_slc.astype(np.float32) / f_slc.max()
a_slc = a_slc.astype(np.float32) / a_slc.max()
m_slc = m_slc.astype(np.float32) / m_slc.max()

# make RGB versions
f_rgb = np.zeros(f_slc.shape + (3,))
f_rgb[..., 0] = f_slc * 2
a_rgb = np.zeros(a_slc.shape + (3,))
a_rgb[..., 0] = f_slc * 2
a_rgb[..., 1] = a_slc * 2
m_rgb = np.zeros(m_slc.shape + (3,))
m_rgb[..., 1] = m_slc * 2

# create figure and subplots
fig = plt.figure(figsize=(12,24))
fig.add_subplot(1, 3, 1)
plt.imshow(f_rgb)
fig.add_subplot(1, 3, 2)
plt.imshow(f_rgb)
plt.imshow(a_rgb)
fig.add_subplot(1, 3, 3)
plt.imshow(m_rgb)
plt.show()

## Deformable alignment
---

Most of the mandatory inputs to `prepare_piecewise_deformable_align` play the same role as they did for affine alignment. There are many additional optional parameters that control the nature of the deformable alignment:

    cc_radius: size of neighborhood around each voxel to compute correlations for image matching term (default 12)
    
    gradient_smoothing: list of length 4, e.g. [a, b, c, d], parameters to gradient field smoothing kernel:
    (a * div(grad) + b * grad(div) + c)^d    default [3., 0., 1., 2.] (for now, b should always be 0)
    
    field_smoothing: list of length 4, e.g. [a, b, c, d], parameters to total field smoothing kernel:
    (a * div(grad) + b * grad(div) + c)^d    default [0.5., 0., 1., 6.] (for now, b should always be 0)
    
    iterations: list, number of iterations at each scale level, default [200, 100]
    
    shrink_factors: list, subsampling factor for each scale level, default [2, 1]
    
    smooth_sigmas: list, Gaussian kernel widths for anti-aliasing filter when sub-sampling at each scale level, default [4., 2.]
    
    step: maximum length each gradient descent step can take in micrometers, default: minimum of fixed image voxel spacings
    
    

In [ ]:
# deform functions are in bistream.deform
from bigstream import deform

# deform the moving image
full_field = deform.prepare_piecewise_deformable_align(
    fix_highres, mov_highres_aligned,
    fix_highres_spacing, fix_highres_spacing,
    blocksize=[256,]*3,
    transpose=[True, False],
    global_affine=global_affine,
    local_affines=local_affines,
    iterations=[2, 1],
)

# sanity check
print(type(full_field))
print(full_field.shape)

In [ ]:
# a write location for the final transform
write_path = './full_highres_transform.zarr'

# execute with defaults
full_field = execute(full_field, write_path=write_path)

## Applying the full transform
---
We provided `global_affine` and `local_affines` to the `prepare_piecewise_deformable_align` function, so the position field it returned includes those transforms. Therefore, `full_field` is the complete transform between the fixed and moving images.

In [ ]:
# apply the local affines to the moving image
mov_highres_aligned = transform.prepare_apply_position_field(
    fix_highres, mov_highres,
    fix_highres_spacing, mov_highres_spacing,
    full_field,
    blocksize=[256,]*3,
    transpose=[True, True, False],
)

# sanity check
print(type(mov_highres_aligned))
print(mov_highres_aligned.shape)

In [ ]:
# a write path for the final result
write_path = './mov_highres_deform_aligned.zarr'

# execute using defaults
mov_highres_aligned = execute(mov_highres_aligned, write_path=write_path, project='multifish')

In [ ]:
# plot some image slices to check on things
slc = 140
f_slc = fix_highres[slc, ...].transpose(1,0)
a_slc = mov_highres_aligned[..., slc]
m_slc = mov_highres[slc, ...].transpose(1,0)

# normalize for display
f_slc = f_slc.astype(np.float32) / f_slc.max()
a_slc = a_slc.astype(np.float32) / a_slc.max()
m_slc = m_slc.astype(np.float32) / m_slc.max()

# make RGB versions
f_rgb = np.zeros(f_slc.shape + (3,))
f_rgb[..., 0] = f_slc * 2
a_rgb = np.zeros(a_slc.shape + (3,))
a_rgb[..., 0] = f_slc * 2
a_rgb[..., 1] = a_slc * 2
m_rgb = np.zeros(m_slc.shape + (3,))
m_rgb[..., 1] = m_slc * 2

# create figure and subplots
fig = plt.figure(figsize=(12,24))
fig.add_subplot(1, 3, 1)
plt.imshow(f_rgb)
fig.add_subplot(1, 3, 2)
plt.imshow(f_rgb)
plt.imshow(a_rgb)
fig.add_subplot(1, 3, 3)
plt.imshow(m_rgb)
plt.show()